# BDII -- Sesión 1 -- Procesado inicial de datos

Esta hoja muestra cómo procesar o curar un conjunto de datos para hacerlos más accesibles a la hora de introducirlos en bases de datos. Utilizaremos un conjunto de datos existente en Internet, que se descargará, se procesará y se convertirá en un formato universal como CSV o JSON. En particular se trabajará:

- La descarga de los datos
- Inspección, identificación del formato y posible procesado
- Generación de un formato fácilmente digerible por las BBDD, como CSV o JSON

Comenzaremos instalando los paquetes necesarios:

In [29]:
!sudo apt-get update -qq

In [30]:
!sudo apt-get install -y git p7zip

Importamos algunos paquetes estándar para la hoja

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

C:\Users\fcoja\AppData\Local\Temp\ipykernel_12032\1979311418.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

## Datos de Stackoverflow

## Descarga de los datos

En este caso los datos están disponibles en un repositorio git. Se pueden descargar también de la Web, pero se van actualizando. En este caso los descargamos del repositorio git para que todos tengáis los mismos.

In [3]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.002
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.003

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [34]:
!ls -l

total 6222364
-rw-r--r-- 1 root root  215792978 Dec  4 13:19 Comments.xml
-rw-r--r-- 1 root root  103809024 Jan 31 16:10 es.stackoverflow.7z.001
-rw-r--r-- 1 root root  103809024 Jan 31 16:41 es.stackoverflow.7z.001.1
-rw-r--r-- 1 root root  103809024 Jan 31 16:10 es.stackoverflow.7z.002
-rw-r--r-- 1 root root  103809024 Jan 31 16:41 es.stackoverflow.7z.002.1
-rw-r--r-- 1 root root   10263728 Jan 31 16:10 es.stackoverflow.7z.003
-rw-r--r-- 1 root root   10263728 Jan 31 16:41 es.stackoverflow.7z.003.1
-rw-r--r-- 1 root root 1212084099 Jan 31 16:14 Posts2.json
-rw-r--r-- 1 root root  965533797 Jan 31 16:12 Posts.csv
-rw-r--r-- 1 root root 1221848946 Jan 31 16:13 Posts.json
-rw-r--r-- 1 root root 1141094239 Jan 31 16:14 Posts.jsonl
-rw-r--r-- 1 root root 1029736014 Dec  4 13:20 Posts.xml
drwxr-xr-x 1 root root       4096 Jan 29 14:26 sample_data
-rw-r--r-- 1 root root     227980 Dec  4 13:20 Tags.xml
-rw-r--r-- 1 root root   76430831 Dec  4 13:20 Users.xml
-rw-r--r-- 1 root root   7314912

In [ ]:
!7zr x es.stackoverflow.7z.001


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 103809024 bytes (99 MiB)

Extracting archive: es.stackoverflow.7z.001
  0% 1 Open           --
Path = es.stackoverflow.7z.001
Type = Split
Physical Size = 103809024
Volumes = 3
Total Physical Size = 217881776
----
Path = es.stackoverflow.7z
Size = 217881776
--
Path = es.stackoverflow.7z
Type = 7z
Physical Size = 217881776
Headers Size = 244
Method = LZMA2:24
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./Comments.xml
  Size:     215792978 bytes (206 MiB)
  Modified: 2023-12-04 13:19:25
with the file from archive:
  Path:     Comments.xml
  Size:     215792978 bytes (206 MiB)
  Modified: 2023-12-04 13:19:25
? (Y)es / (N)o / (A)lways / (S)kip

In [ ]:
!ls -l

## Inspección y procesado

Podemos inspeccionar los ficheros `.xml` para ver su contenido. Son XML, sí, pero ¿con qué formato?

In [6]:
!head Posts.xml

"head" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Aunque se puede procesar el formato XML, lo que podemos ver es que cada entrada es exactamente una línea que comienza por `<row`, y que contiene un conjunto de atributos en formato `atributo="valor"`. Si lo comprobamos, incluso no existirá ninguna comilla doble **dentro** de otra comilla doble, así que podemos extraer esos pares de forma facil.

In [4]:
import re

def process_file(fname):
  with open(fname, "r") as f:
    for line in f:
      if "<row" in line:
        attr_dict = {}
        (_,attrs) = line.split("<row ",2)
        for m in re.finditer(r"(\w*?)=\"(.*?)\"", attrs):
          attr_dict[m.group(1)] = m.group(2)
        yield attr_dict

In [5]:
first_row = next(process_file("Posts.xml"))

FileNotFoundError: [Errno 2] No such file or directory: 'Posts.xml'

In [ ]:
first_row.keys()

Hay que extraer el conjunto de atributos para saber qué columnas tendrá nuestra tabla/CSV o archivo JSON. Recuérdese que las dos primeras filas del archivo XML tenían diferentes atributos. ¿Cómo se haría esto?

In [ ]:
def get_all_attrs(iterator):
  all_attrs = set()
  for row in iterator:
    all_attrs.update(row.keys())
  return all_attrs

all_attrs = get_all_attrs(process_file("Posts.xml"))

Como sabemos que el atributo `Id` va a ser la clave primaria, lo ponemos al principio. Además, generamos una lista, uno un conjunto, para que el orden sea conocido.

In [ ]:
all_attrs.remove('Id')
all_attrs = list(all_attrs)
all_attrs.insert(0,'Id')
all_attrs

## Escritura del formato CSV

El formato CSV está especificado en el estándar RFC 4180. https://www.ietf.org/rfc/rfc4180.txt. En general se puede utilizar la biblioteca `csv` de Python 3 y vamos a exportar una línea de cabecera con todos los campos. https://docs.python.org/3/library/csv.html.

Tendremos en cuenta que todas las filas tienen que tener las mismas columnas y en el mismo orden dado por `all_attrs`.

In [ ]:
import csv

def write_csv(destfile, all_attrs, iterator):
  with open(destfile, 'w') as wf:
    cw = csv.writer(wf)

    # Escribir la línea de cabecera
    cw.writerow(all_attrs)

    # Recorrer el iterador
    for row in iterator:
      cw.writerow(map(lambda att: row.get(att) or '', all_attrs))

In [ ]:
write_csv('Posts.csv', all_attrs, process_file('Posts.xml'))

In [ ]:
!head Posts.csv

In [ ]:
def read_csv(filename):
  with open(fname, "r") as f:
    for line in f:
      yield line.split(',')

## Conversión hacia JSON

https://www.json.org/json-en.html

In [ ]:
import json

def csv_to_json(fname_csv, fname_json, pk):
    data_dict = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for rows in csv_reader:
            key = rows[pk]
            data_dict[key] = rows

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent = 4))

# Estamos cargando todo en la memoria, con conjuntos grandes de datos puede
# resultar muy pesado.

In [ ]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json(fname_csv, fname_json, 'Id')

In [ ]:
!head Posts.json

Si nos damos cuenta, tenemos el problema de que el valor Id está por duplicado.

Vamos a ver como eliminar columnas que no queramos tener.

In [ ]:
def csv_to_json2(fname_csv, fname_json, pk):
    data_dict = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for rows in csv_reader:
            key = rows[pk]

            # Borramos los campos que nos interesen.
            del rows[pk]

            data_dict[key] = rows

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent = 4))

In [ ]:
fname_csv = 'Posts.csv'
fname_json = 'Posts2.json'

csv_to_json2(fname_csv, fname_json, 'Id')

In [ ]:
!head -n 100 Posts2.json

Al escribir en formato JSON se nos queda un fichero compacto que no podemos dividir.

## JSON Lines



https://jsonlines.org/


In [ ]:
def csv_to_jsonl(fname_csv, fname_jsonl):
    with open(fname_csv, 'r') as f_csv:
        csv_reader = csv.DictReader(f_csv)

        with open(fname_jsonl, 'w') as f_jsonl:
            for row in csv_reader:
                json_line = json.dumps(row)
                f_jsonl.write(json_line + '\n')

In [ ]:
csv_to_jsonl('Posts.csv', 'Posts.jsonl')

In [ ]:
!head Posts.jsonl